In [2]:
#!pip install pycoingecko
!pip install kafka-python

     |████████████████████████████████| 246 kB 3.3 MB/s eta 0:00:01


In [6]:
import pandas as pd
from pycoingecko import CoinGeckoAPI
from kafka import KafkaConsumer, KafkaProducer

In [15]:
def get_crypto_data():
    
    crypto_list = [
        #"bitcoin",
        # "ethereum",
        # "ripple",
        # "bitcoin-cash",
         "cardano"
        # "bitcoin-cash-sv",
        # "litecoin",
        # "chainlink",
        # "binancecoin",
        # "eos",
        # "tron",
    ]
    
    #connect to API
    api = CoinGeckoAPI()
    timePeriod = 20 #data backed to about 6/1/2020
    
    #connect to kafka
    producer = KafkaProducer(bootstrap_servers='localhost:9092',api_version=(0,11,5))
    topic_name='crypto_data'

    #getting crypto prices 
    data = {}
    for coin in crypto_list:
        try:
            nested_lists = api.get_coin_market_chart_by_id(id=coin, vs_currency="usd", days=timePeriod)["prices"]
            data[coin] = {}
            #groups time and price to each coin
            data[coin]["timestamps"], data[coin]["values"] = zip(*nested_lists)

        except Exception as e:
            print(e)
            print("coin: " + coin)

    frame_list = [
        pd.DataFrame(data[coin]["values"], index=data[coin]["timestamps"], columns=[coin])
        for coin in crypto_list
        if coin in data
    ]
    
    #formatting dataset
    df_crypto = pd.concat(frame_list, axis=1).sort_index()
    df_crypto["datetime"] = pd.to_datetime(df_crypto.index, unit="ms")
    df_crypto["date"] = df_crypto["datetime"].dt.date
    df_crypto["time"] = df_crypto["datetime"].dt.time
    df_crypto = df_crypto.melt(id_vars=["datetime", "date", "time"], var_name="currency_name", ignore_index=True)
    df_crypto.dropna(inplace=True)
    
    #adding symbol to dataset
    curr = dict(
        [
            #("bitcoin", "BTC"),
            # ("ethereum", "ETH"),
            # ("ripple", "XRP"),
            # ("bitcoin-cash", "BCH"),
             ("cardano", "ADA")
            # ("bitcoin-cash-sv", "BSV"),
            # ("litecoin", "LTC"),
            # ("chainlink", "LINK"),
            # ("binancecoin", "BNB"),
            # ("tron", "TRON"),
            # ("eos", "EOS"),
            # ("tether", "USDT"),
        ]
    )

    df_crypto["coin_symbol"] = df_crypto["currency_name"].apply(lambda c: curr[c])
    
    #convert df to json
    #json = df_crypto.to_json()

    #sending to formatted dataset to kafka
    producer.send(topic_name, df_crypto)
    

In [17]:
get_crypto_data()

AssertionError: 